Timing
------

Quickly time a single line.

In [154]:
import math
import ubelt as ub
timer = ub.Timer('Timer demo!', verbose=1)
with timer:
    math.factorial(100000)


tic('Timer demo!')
...toc('Timer demo!')=0.0933s


Loop Progress
-------------

``ProgIter`` is a (mostly) drop-in alternative to
```tqdm`` <https://pypi.python.org/pypi/tqdm>`__. 
*The advantage of ``ProgIter`` is that it does not use any python threading*,
and therefore can be safer with code that makes heavy use of multiprocessing.

Note: ProgIter is now a standalone module: ``pip intstall progiter``)

In [155]:
import ubelt as ub
import math
for n in ub.ProgIter(range(7500)):
     math.factorial(n)

 7500/7500... rate=3321.68 Hz, eta=0:00:00, total=0:00:0200


In [156]:
import ubelt as ub
import math
for n in ub.ProgIter(range(7500), freq=1000, adjust=False):
     math.factorial(n)
        
# Note that forcing freq=2 all the time comes at a performance cost
# The default adjustment algorithm causes almost no overhead

 7500/7500... rate=3305.80 Hz, eta=0:00:00, total=0:00:0200


In [157]:
>>> import ubelt as ub
>>> def is_prime(n):
...     return n >= 2 and not any(n % i == 0 for i in range(2, n))
>>> for n in ub.ProgIter(range(1000), verbose=2):
>>>     # do some work
>>>     is_prime(n)

    0/1000... rate=0 Hz, eta=?, total=0:00:00
    1/1000... rate=126214.91 Hz, eta=0:00:00, total=0:00:00
    4/1000... rate=16279.95 Hz, eta=0:00:00, total=0:00:00
   16/1000... rate=35893.59 Hz, eta=0:00:00, total=0:00:00
   64/1000... rate=88745.17 Hz, eta=0:00:00, total=0:00:00
  256/1000... rate=201950.56 Hz, eta=0:00:00, total=0:00:00
 1000/1000... rate=183798.63 Hz, eta=0:00:00, total=0:00:00


Caching
-------

Cache intermediate results from blocks of code inside a script with minimal
boilerplate or modification to the original code.  

For direct caching of data, use the ``Cacher`` class.  By default results will
be written to the ubelt's appdir cache, but the exact location can be specified
via ``dpath`` or the ``appname`` arguments.  Additionally, process dependencies
can be specified via the ``depends`` argument, which allows for implicit cache
invalidation.  As far as I can tell, this is the most concise way (4 lines of
boilerplate) to cache a block of code with existing Python syntax (as of
2022-06-03).


In [158]:
import ubelt as ub
depends = ['config', {'of': 'params'}, 'that-uniquely-determine-the-process']
cacher = ub.Cacher('test_process', depends=depends, appname='myapp', verbose=3)

if 1:
    cacher.fpath.delete()
    
for _ in range(2):
    data = cacher.tryload()
    if data is None:
        myvar1 = 'result of expensive process'
        myvar2 = 'another result'
        data = myvar1, myvar2
        cacher.save(data)
    myvar1, myvar2 = data

[cacher] tryload fname=test_process
[cacher] ... cache does not exist: dpath=myapp fname=test_process cfgstr=66783cfd3e2c9799bb98f5eec57738915ec3777be02e395a0e10ad566c07f2c25876fd1edd4f4fc2280601cae3c09efe539f18f2c5a7bb954764786f5be4b72b
[cacher] ... test_process cache miss
[cacher] ... test_process cache save
[cacher] tryload fname=test_process
[cacher] ... test_process cache hit


For indirect caching, use the ``CacheStamp`` class. This simply writes a
"stamp" file that marks that a process has completed. Additionally you can
specify criteria for when the stamp should expire. If you let ``CacheStamp``
know about the expected "product", it will expire the stamp if that file has
changed, which can be useful in situations where caches might becomes corrupt
or need invalidation.

In [159]:
import ubelt as ub
dpath = ub.Path.appdir('ubelt/demo/cache').delete().ensuredir()
params = {'params1': 1, 'param2': 2}
expected_fpath = dpath / 'file.txt'
stamp = ub.CacheStamp('name', dpath=dpath, depends=params,
                     hasher='sha256', product=expected_fpath,
                     expires='2101-01-01T000000Z', verbose=3)

if 1:
    stamp.fpath.delete()
    
for _ in range(2):
    if stamp.expired():
        expected_fpath.write_text('expensive process')
        stamp.renew()

[cacher] tryload fname=name
[cacher] ... cache does not exist: dpath=cache fname=name cfgstr=4a166a5cbaa2926ccceb1620ee63fa1e3c4626229e887c7604b88b44e5f5df021e172437c359614dfdce1be2043909aa54194da54b6bd20b9e1f558b48756a26
[cacher] ... name cache miss
[cacher] stamp expired no_cert
[cacher] ... name cache save
[cacher] tryload fname=name
[cacher] ... name cache hit


Hashing
-------

The ``ub.hash_data`` constructs a hash for common Python nested data
structures. Extensions to allow it to hash custom types can be registered.  By
default it handles lists, dicts, sets, slices, uuids, and numpy arrays.

In [160]:
import ubelt as ub
data = [('arg1', 5), ('lr', .01), ('augmenters', ['flip', 'translate'])]
ub.hash_data(data, hasher='sha256')

'0d95771ff684756d7be7895b5594b8f8484adecef03b46002f97ebeb1155fb15'

Support for torch tensors and pandas data frames are also included, but needs to
be explicitly enabled.  There also exists an non-public plugin architecture to
extend this function to arbitrary types. While not officially supported, it is
usable and will become better integrated in the future. See
``ubelt/util_hash.py`` for details.

Command Line Interaction
------------------------

The builtin Python ``subprocess.Popen`` module is great, but it can be a
bit clunky at times. The ``os.system`` command is easy to use, but it
doesn't have much flexibility. The ``ub.cmd`` function aims to fix this.
It is as simple to run as ``os.system``, but it returns a dictionary
containing the return code, standard out, standard error, and the
``Popen`` object used under the hood.

In [161]:
import ubelt as ub
info = ub.cmd('cmake --version')
# Quickly inspect and parse output of a 
print(info['out'])

cmake version 3.22.1

CMake suite maintained and supported by Kitware (kitware.com/cmake).



In [162]:
# The info dict contains other useful data
print(ub.repr2({k: v for k, v in info.items() if 'out' != k}))

{
    'command': 'cmake --version',
    'cwd': None,
    'err': '',
    'proc': <Popen: returncode: 0 args: ['cmake', '--version']>,
    'ret': 0,
}


In [163]:
# Also possible to simultaneously capture and display output in realtime
info = ub.cmd('cmake --version', tee=1)

cmake version 3.22.1

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [164]:
# tee=True is equivalent to using verbose=1, but there is also verbose=2
info = ub.cmd('cmake --version', verbose=2)

[ubelt.cmd] joncrall@toothbrush:~/code/ubelt/docs/notebooks$ cmake --version
cmake version 3.22.1

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [165]:
# and verbose=3
info = ub.cmd('cmake --version', verbose=3)

┌─── START CMD ───
[ubelt.cmd] joncrall@toothbrush:~/code/ubelt/docs/notebooks$ cmake --version
cmake version 3.22.1

CMake suite maintained and supported by Kitware (kitware.com/cmake).
└─── END CMD ───


Cross-Platform Config and Cache Directories
-------------------------------------------

If you have an application which writes configuration or cache files,
the standard place to dump those files differs depending if you are on
Windows, Linux, or Mac. Ubelt offers a unified functions for determining
what these paths are.

The ``ub.ensure_app_cache_dir`` and ``ub.ensure_app_config_dir``
functions find the correct platform-specific location for these files
and ensures that the directories exist. (Note: replacing "ensure" with
"get" will simply return the path, but not ensure that it exists)

The config root directory is ``~/AppData/Roaming`` on Windows,
``~/.config`` on Linux and ``~/Library/Application Support`` on Mac. The
cache root directory is ``~/AppData/Local`` on Windows, ``~/.config`` on
Linux and ``~/Library/Caches`` on Mac.

Example usage on Linux might look like this:

In [166]:
import ubelt as ub
print(ub.shrinkuser(ub.ensure_app_cache_dir('my_app')))
print(ub.shrinkuser(ub.ensure_app_config_dir('my_app')))

~/.cache/my_app
~/.config/my_app


New in version 1.0.0: the ``ub.Path.appdir`` classmethod provides a way to
achieve the above with a chainable object oriented interface.

In [167]:
import ubelt as ub
print(ub.Path.appdir('my_app').ensuredir().shrinkuser())
print(ub.Path.appdir('my_app', type='config').ensuredir().shrinkuser())

~/.cache/my_app
~/.config/my_app


Downloading Files
-----------------

The function ``ub.download`` provides a simple interface to download a
URL and save its data to a file.

The function ``ub.grabdata`` works similarly to ``ub.download``, but
whereas ``ub.download`` will always re-download the file,
``ub.grabdata`` will check if the file exists and only re-download it if
it needs to.

New in version 0.4.0: both functions now accepts the ``hash_prefix`` keyword
argument, which if specified will check that the hash of the file matches the
provided value. The ``hasher`` keyword argument can be used to change which
hashing algorithm is used (it defaults to ``"sha512"``).

In [168]:
    >>> import ubelt as ub
    >>> url = 'http://i.imgur.com/rqwaDag.png'
    >>> fpath = ub.download(url, verbose=0)
    >>> print(ub.shrinkuser(fpath))

~/.cache/ubelt/rqwaDag.png


In [169]:
    >>> import ubelt as ub
    >>> url = 'http://i.imgur.com/rqwaDag.png'
    >>> fpath = ub.grabdata(url, verbose=0, hash_prefix='944389a39')
    >>> print(ub.shrinkuser(fpath))

~/.cache/ubelt/rqwaDag.png


In [170]:
url = 'http://i.imgur.com/rqwaDag.png'
ub.grabdata(url, verbose=3, hash_prefix='944389a39dfb8f')

try:
    ub.grabdata(url, verbose=3, hash_prefix='wrong-944389a39dfb8f')
except RuntimeError as ex:
    print('type(ex) = {!r}'.format(type(ex)))

[cacher] tryload fname=rqwaDag.png.stamp
[cacher] ... rqwaDag.png.stamp cache hit
[cacher] tryload fname=rqwaDag.png.stamp
[cacher] ... rqwaDag.png.stamp cache hit
invalid hash prefix value (expected "wrong-944389a39dfb8f", got "944389a39dfb8fa9e3d075bc25416d56782093d5dca88a1f84cac16bf515fa12aeebbbebf91f1e31e8beb59468a7a5f3a69ab12ac1e3c1d1581e1ad9688b766f")
invalid hash prefix value (expected "wrong-944389a39dfb8f", got "944389a39dfb8fa9e3d075bc25416d56782093d5dca88a1f84cac16bf515fa12aeebbbebf91f1e31e8beb59468a7a5f3a69ab12ac1e3c1d1581e1ad9688b766f")
 1233/1233... rate=2669535.98 Hz, eta=0:00:00, total=0:00:00
hash_prefix = 'wrong-944389a39dfb8f'
got = '944389a39dfb8fa9e3d075bc25416d56782093d5dca88a1f84cac16bf515fa12aeebbbebf91f1e31e8beb59468a7a5f3a69ab12ac1e3c1d1581e1ad9688b766f'
type(ex) = <class 'RuntimeError'>


# Dictionary Tools

In [171]:
import ubelt as ub
items    = ['ham',     'jam',   'spam',     'eggs',    'cheese', 'bannana']
groupids = ['protein', 'fruit', 'protein',  'protein', 'dairy',  'fruit']
groups = ub.group_items(items, groupids)
print(ub.repr2(groups, nl=1))

{
    'dairy': ['cheese'],
    'fruit': ['jam', 'bannana'],
    'protein': ['ham', 'spam', 'eggs'],
}


In [172]:
import ubelt as ub
items = [1, 2, 39, 900, 1232, 900, 1232, 2, 2, 2, 900]
ub.dict_hist(items)

{1: 1, 2: 4, 39: 1, 900: 3, 1232: 2}

In [173]:
import ubelt as ub
items = [0, 0, 1, 2, 3, 3, 0, 12, 2, 9]
ub.find_duplicates(items, k=2)

{0: [0, 1, 6], 2: [3, 8], 3: [4, 5]}

In [174]:
import ubelt as ub
dict_ = {'K': 3, 'dcvs_clip_max': 0.2, 'p': 0.1}
subdict_ = ub.dict_subset(dict_, ['K', 'dcvs_clip_max'])
print(subdict_)

OrderedDict([('K', 3), ('dcvs_clip_max', 0.2)])


In [175]:
import ubelt as ub
dict_ = {1: 'a', 2: 'b', 3: 'c'}
print(list(ub.take(dict_, [1, 3, 4, 5], default=None)))

['a', 'c', None, None]


In [176]:
import ubelt as ub
dict_ = {'a': [1, 2, 3], 'b': []}
newdict = ub.map_vals(len, dict_)
print(newdict)

{'a': 3, 'b': 0}


In [177]:
import ubelt as ub
mapping = {0: 'a', 1: 'b', 2: 'c', 3: 'd'}
ub.invert_dict(mapping)

{'a': 0, 'b': 1, 'c': 2, 'd': 3}

In [178]:
import ubelt as ub
mapping = {'a': 0, 'A': 0, 'b': 1, 'c': 2, 'C': 2, 'd': 3}
ub.invert_dict(mapping, unique_vals=False)

{0: {'A', 'a'}, 1: {'b'}, 2: {'C', 'c'}, 3: {'d'}}

AutoDict - Autovivification
---------------------------

While the ``collections.defaultdict`` is nice, it is sometimes more
convenient to have an infinitely nested dictionary of dictionaries.

(But be careful, you may start to write in Perl) 

In [179]:
>>> import ubelt as ub
>>> auto = ub.AutoDict()
>>> print('auto = {!r}'.format(auto))
>>> auto[0][10][100] = None
>>> print('auto = {!r}'.format(auto))
>>> auto[0][1] = 'hello'
>>> print('auto = {!r}'.format(auto))

auto = {}
auto = {0: {10: {100: None}}}
auto = {0: {10: {100: None}, 1: 'hello'}}


String-based imports
--------------------

Ubelt contains functions to import modules dynamically without using the
python ``import`` statement. While ``importlib`` exists, the ``ubelt``
implementation is simpler to user and does not have the disadvantage of
breaking ``pytest``.

Note ``ubelt`` simply provides an interface to this functionality, the
core implementation is in ``xdoctest`` (over as of version ``0.7.0``, 
the code is statically copied into an autogenerated file such that ``ubelt``
does not actually depend on ``xdoctest`` during runtime).

In [180]:
import ubelt as ub
try:
    # This is where I keep ubelt on my machine, so it is not expected to work elsewhere.
    module = ub.import_module_from_path(ub.expandpath('~/code/ubelt/ubelt'))
    print('module = {!r}'.format(module))
except OSError:
    pass
        
module = ub.import_module_from_name('ubelt')
print('module = {!r}'.format(module))

try:
    module = ub.import_module_from_name('does-not-exist')
    raise AssertionError
except ModuleNotFoundError:
    pass

modpath = ub.Path(ub.util_import.__file__)
print(ub.modpath_to_modname(modpath))
modname = ub.util_import.__name__
assert ub.Path(ub.modname_to_modpath(modname)).resolve() == modpath.resolve()

module = <module 'ubelt' from '/home/joncrall/code/ubelt/ubelt/__init__.py'>
module = <module 'ubelt' from '/home/joncrall/code/ubelt/ubelt/__init__.py'>
ubelt.util_import


Related to this functionality are the functions
``ub.modpath_to_modname`` and ``ub.modname_to_modpath``, which
*statically* transform (i.e. no code in the target modules is imported
or executed) between module names (e.g. ``ubelt.util_import``) and
module paths (e.g.
``~/.local/conda/envs/cenv3/lib/python3.5/site-packages/ubelt/util_import.py``).

Horizontal String Concatenation
-------------------------------

Sometimes its just prettier to horizontally concatenate two blocks of
text.

In [181]:
    >>> import ubelt as ub
    >>> B = ub.repr2([[1, 2], [3, 4]], nl=1, cbr=True, trailsep=False)
    >>> C = ub.repr2([[5, 6], [7, 8]], nl=1, cbr=True, trailsep=False)
    >>> print(ub.hzcat(['A = ', B, ' * ', C]))

A = [[1, 2], * [[5, 6],
     [3, 4]]    [7, 8]]
